In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
from time import time

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', 65)  # default: 50

train = pd.read_csv("train_preprocessed.csv")


In this nb we'll play around with positive examples, trying to infer the possible types of questions available. 

First, let's drop all the negative examples. 

In [2]:
train.columns

Index(['Unnamed: 0', 'id', 'qid1', 'qid2', 'question1', 'question2',
       'is_duplicate'],
      dtype='object')

In [3]:
train.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,what is the step by step guide to invest in share market in i...,what is the step by step guide to invest in share market?,0
1,1,1,3,4,what is the story of kohinoor (koh-i-noor) diamond?,what would happen if the indian government stole the kohinoor...,0
2,2,2,5,6,how can i increase the speed of my internet connection while ...,how can internet speed be increased by hacking through dns?,0
3,4,4,9,10,"which one dissolve in water quikly sugar, salt, methane and c...",which fish would survive in salt water?,0
4,5,5,11,12,astrology: i am a capricorn sun cap moon and cap rising...wha...,"i'm a triple capricorn (sun, moon and ascendant in capricorn)...",1


In [4]:
train = train.loc[ train.is_duplicate == 1 , lambda df: ['question1', 'question2'] ] 

We'll now change the questions to be lists of words, and remove all words below some frequency. This should leave only the interesting words (the correct thing to do is probably use them as well as word2vec, so that some of the meaning remains)

In [5]:
words_lst_iterator1 = itertools.chain(*[''.join(map(lambda x: x if str.isalnum(x) else ' ', q)).split() for q in train.question1])
words_lst_iterator2 = itertools.chain(*[''.join(map(lambda x: x if str.isalnum(x) else ' ', q)).split() for q in train.question2])

In [6]:
from collections import Counter
word_counter = Counter(itertools.chain(words_lst_iterator1, words_lst_iterator2))
words = sorted(word_counter.keys(), key = lambda x: word_counter[x], reverse = True)

In [31]:
q_words = [p[0] for p in word_counter.most_common()[:59]]

In [25]:
word_counter["than"]

2825

In [102]:
%pprint
q_words

Pretty printing has been turned OFF


['the', 'what', 'is', 'how', 'i', 'to', 'do', 'in', 'a', 'are', 'of', 'can', 'and', 'you', 'best', 'why', 'for', 'my', 'on', 'it', 'which', 'be', 'some', 'does', 'your', 'get', 'if', 's', 'india', 'should', 'that', 'will', 'quora', 'have', 'or', 'with', 'people', 'from', 'an', 'who', 'money', 'way', 'would', 'life', 'good', 'about', 'there', 'make', 'most', 'trump', 'we', 'learn', 'when', 'one', 'like', 'did', 'as', 'between', '500']

In [135]:
def word_in_qwords(w, q_words = q_words):
    if w in q_words:
        return w
    else:
        return "*"
    
def jokerize_question(question):
    q = (''.join(map(lambda x: x if str.isalnum(x) else ' ', question))).split()
    q = list(map( word_in_qwords, q))
    l = []
    for i in range(len(q)-1):
        if q[i] == q[i+1] == '*':
            continue
        else:
            l.append(q[i])
    return l

In [144]:
preprocessed = train['question1'].apply(jokerize_question)

KeyError: ('question1', 'question2')

In [139]:
tf.contrib.rnn.BasicLSTMCell()

In [146]:
train.columns

Index(['question1', 'question2'], dtype='object')

In [147]:
pped_data = train.applymap(jokerize_question)

In [148]:
pped_data.head()

,question1,question2
4,"[*, i, *, a, *, and, *, what, does, that, *, about]","[i, *, a, *, and, *, in, *, what, does, *, about]"
6,"[how, can, i, be, a, good]","[what, should, i, do, to, be, a]"
9,"[how, do, i, *, and, *, my]","[how, can, i, *, my]"
10,"[what, can, make, *, to]","[how, can, you, make, *, to]"
11,"[what, *, your, *]","[what, *, your]"


In [151]:
len(q_words)

59

In [153]:
q_words.append(['*'])

In [154]:
len(q_words)

60

In [155]:
tf.one_hot(pped_data.iloc[1:5]['question1'], 20)

ValueError: Argument must be a dense tensor: 6      [how, can, i, be, a, good]
9     [how, do, i, *, and, *, my]
10       [what, can, make, *, to]
11             [what, *, your, *]
Name: question1, dtype: object - got shape [4], but wanted [].

In [156]:
import keras

Using TensorFlow backend.
